In [2]:
# import statements
import numpy as np
import dicom
import cv2 
import matplotlib.pyplot as plt

extract data

In [3]:
base = '/home/radiation/RadiationTherapyDecisionSupport/data/'
file_folder = 'UCLA_PR_5/'

structureset = dicom.read_file(base + file_folder + 'structureset.dcm')

In [4]:
dose_data = dicom.read_file(base + file_folder + "dose.dcm")

In [5]:
dose_grid = np.array(dose_data.pixel_array)

In [6]:
dose_grid = np.swapaxes(np.swapaxes(dose_grid, 0, 2), 0, 1)
print(dose_grid.shape)

(136, 206, 317)


In [7]:
dose_grid = dose_grid * dose_data.DoseGridScaling

In [8]:
dose_grid = np.expand_dims(dose_grid, axis=2)
print(dose_grid.shape)

(136, 206, 1, 317)


In [9]:
dose_grid = np.repeat(dose_grid, 3, axis=2)
print(dose_grid.shape)

(136, 206, 3, 317)


In [10]:
maxDose = np.max(dose_grid)

In [11]:
dose_grid = dose_grid/maxDose
#print(np.unique(dose_grid))

In [12]:
isodoseValues = np.array([40, 50, 60, 70, 80, 90, 95])

In [13]:
doseBlocks = np.zeros(dose_grid.shape).astype(np.uint8)

In [ ]:
for n in range(0, 1): #len(isodoseValues)):
    tempDoseMask = np.zeros(dose_grid.shape).astype(np.uint8)
    doseOutline = np.zeros(dose_grid.shape).astype(np.uint8)
    tempDoseMask[dose_grid > isodoseValues[n]*0.01] = 1 #removed maxDose factor
    #print(np.unique(tempDoseMask))
    #tempDoseGrid = np.array(dose_grid[:, :, channel, j].astype(np.float64))
    #print(np.unique(tempDoseGrid))
    
    for j in range(0, dose_grid.shape[3]):
        for channel in range(0, 3):
            doseOutline[:,:,channel,j], contours, hierarchy = cv2.findContours(tempDoseMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
            
            temp_array = np.zeros(doseOutline[:,:,channel,j].shape).astype(np.uint8)
            
            cv2.drawContours(temp_array, contours, -1, 255, 1)
            temp_array[temp_array==255] = isodoseValues[n]
            doseBlocks[:,:,channel,j] += temp_array;   

In [ ]:
plt.imshow(doseBlocks[:,:,:,0], cmap=plt.cm.gray)
plt.show()

In [ ]:
print(np.unique(doseBlocks))